## Tarea de Redes Nueronales.
## Alumno: Federico Daniel Moran Fretes

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [3]:
data = pd.read_csv("Dataset\mnist_train_small.csv", header = None)
data_test = pd.read_csv("Dataset\mnist_test.csv", header = None)

<>:1: SyntaxWarning: invalid escape sequence '\m'
<>:2: SyntaxWarning: invalid escape sequence '\m'
<>:1: SyntaxWarning: invalid escape sequence '\m'
<>:2: SyntaxWarning: invalid escape sequence '\m'
C:\Users\moran\AppData\Local\Temp\ipykernel_22632\2292152679.py:1: SyntaxWarning: invalid escape sequence '\m'
  data = pd.read_csv("Dataset\mnist_train_small.csv", header = None)
C:\Users\moran\AppData\Local\Temp\ipykernel_22632\2292152679.py:2: SyntaxWarning: invalid escape sequence '\m'
  data_test = pd.read_csv("Dataset\mnist_test.csv", header = None)


In [5]:
X = data.iloc[:,1:]
X_test = data_test.iloc[:,1:]

In [6]:
X = X.astype("float32")/255
X_test = X_test.astype("float32")/255

In [7]:
X_test.shape

(10000, 784)

# Etapa 1 - Autoencoder

In [8]:
from tensorflow import keras
from tensorflow.keras import layers, models